# Chapter 5. Support Vector Machines

## Setup

In [1]:
# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# to plot figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

## Linear SVM Classification